In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar a página da Alura:** Fique de olho na página principal da Alura ([https://www.alura.com.br/](https://www.alura.com.br/)) e nas redes sociais deles (LinkedIn, Instagram, etc.). Eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de Inteligência Artificial da Alura:** Confira a seção de Inteligência Artificial no site da Alura, onde você pode encontrar informações sobre os cursos e imersões relacionados.
*   **Inscrever-se na newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre os lançamentos e novidades da plataforma.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Imersão IA com Google Gemini da Alura ocorreu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram encerradas em 11 de maio de 2025.

No momento, não tenho informações sobre a data da próxima edição da Imersão IA com Google Gemini da Alura. Para ficar por dentro das próximas edições, você pode acompanhar as redes sociais da Alura e assinar a newsletter para receber novidades.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: tecmundo.com.br, alura.com.br, youtube.com



In [7]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00


In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Cria um serviço de sessão em memória
session_service = InMemorySessionService()

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [12]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
async def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [13]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
async def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = await call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
async def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = await call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [15]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = await agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: IA
Maravilha! Vamos então criar o post sobre novidades em IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos de IA mais relevantes e recentes (dentro do último mês), vou usar o Google para pesquisar notícias e artigos sobre o tema. O foco será em identificar os lançamentos que geraram mais cobertura e entusiasmo.
> 
> 
> Com base nas notícias e artigos encontrados, aqui estão alguns dos lançamentos e tendências mais relevantes em IA para junho de 2025:
> 
> 1.  **OpenAI O3 Pro:** A OpenAI lançou o novo modelo O3 Pro, considerado um dos melhores do mundo.
> 
> 2.  **Google Gemini:** O Google está investindo pesado em IA, com destaque para o Gemini 2.0 Flash (uma IA ultrarrápida). A empresa também apresentou o Gemma 3N, um modelo de IA menor com potencial para uso em dispositivos móveis.
> 
> 3.  **Avanços em Agentes Autônomos de IA:** Empresas como Amazon, Google (com o Project Astra) e Apple estão desenvolvendo agentes de IA autônomos capazes de realizar tarefas complexas de forma independente. Esses agentes poderão acessar sistemas internos e dados proprietários, automatizando tarefas como gestão de calendários e suporte ao cliente.
> 
> 4.  **Microsoft Copilot:** O Microsoft Copilot recebeu atualizações, incluindo a capacidade de gerar imagens utilizando as melhorias do ChatGPT.
> 
> 5.  **IA Generativa para Vídeos:** A produção de vídeos gerados por IA está evoluindo, com avanços em física computacional e algoritmos de consistência visual permitindo a criação de vídeos mais longos e fluidos.
> 
> Além desses lançamentos específicos, algumas tendências gerais em IA para 2025 incluem:
> 
> *   **Aumento do uso de chatbots de IA como fonte de informação:** Principalmente entre os mais jovens.
> *   **IA Colaborativa (Human-in-the-Loop):** Uma abordagem em que a inteligência artificial e os humanos trabalham juntos para melhorar os resultados.
> *   **Integração da IA com a Internet das Coisas (IoT):** Dispositivos conectados que podem aprender e agir de forma autônoma.
> *   **Aplicações da IA em Cibersegurança:** Para proteger dados e sistemas contra ameaças.
> *   **Avanços da IA na área da saúde:** A IA está transformando a forma como as empresas lidam com dados.
> 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos e tendências em IA que você listou, vou criar um plano para um post de mídia social focado no tema mais relevante. Para isso, farei algumas pesquisas adicionais para aprofundar meu conhecimento sobre cada tópico.
> 
> 
> Com base nas pesquisas, o tema mais relevante para um post de mídia social sobre IA em junho de 2025 é:
> 
> **TEMA:** Adoção de Chatbots de IA como Fonte de Informação, Principalmente entre Jovens
> 
> **Pontos Relevantes:**
> 
> *   **Crescimento do Uso:** Chatbots como ChatGPT estão se tornando fontes de informação populares, especialmente entre jovens (menores de 35 anos).
> *   **Preferências dos Usuários:** Jovens apreciam a capacidade de chatbots resumirem artigos, traduzirem idiomas, fazerem recomendações e responderem a perguntas sobre a atualidade.
> *   **Desconfiança:** Apesar do crescimento, muitos usuários, incluindo os jovens, ainda desconfiam da confiabilidade das informações geradas por IA e preferem fontes de notícias com supervisão humana.
> *   **Impacto Político:** A IA generativa está sendo usada por figuras políticas para disseminar informações, levantando questões sobre transparência e confiabilidade.
> * Estudo do Instituto Reuters: Estudo recente aponta que 15% dos jovens abaixo de 25 anos utilizam chatbots como fonte primária de informações.
> 
> **Plano para o Post:**
> 
> 1.  **Título Atraente:** Algo como: "A Geração IA: Chatbots Estão Substituindo as Notícias Tradicionais?" ou "Jovens e IA: Uma Nova Era na Busca por Informação (Com um Alerta!)."
> 
> 2.  **Introdução:**
>     *   Começar com uma pergunta que chame a atenção: "Onde você busca suas notícias hoje em dia? Sabia que a IA está se tornando a principal fonte para muitos jovens?"
>     *   Apresentar o crescimento do uso de chatbots como ChatGPT como fonte de informação, com estatísticas sobre a adoção entre jovens (15% dos jovens abaixo de 25 anos).
> 
> 3.  **Corpo do Post:**
>     *   **Vantagens dos Chatbots:**
>         *   Listar as vantagens que atraem os jovens: resumos rápidos, traduções fáceis, recomendações personalizadas.
>         *   Exemplificar como essas ferramentas podem facilitar o acesso à informação para estudantes e pessoas com pouco tempo.
>     *   **A Desconfiança é Real:**
>         *   Abordar a preocupação com a confiabilidade e transparência das informações geradas por IA.
>         *   Citar o estudo do Instituto Reuters que mostra que muitos usuários ainda preferem notícias com supervisão humana.
>         *   Incentivar o pensamento crítico e a verificação de fatos ao usar chatbots para se informar.
>     *   **O Lado Sombrio da IA:**
>         *   Mencionar brevemente como a IA generativa pode ser usada para manipular a opinião pública e disseminar informações falsas.
>         *   Enfatizar a importância de fontes de notícias confiáveis e jornalismo de qualidade.
> 
> 4.  **Chamada para Ação:**
>     *   Incentivar os seguidores a compartilhar suas opiniões: "Você usa chatbots para se informar? Confia nas informações que eles fornecem?"
>     *   Sugerir fontes de notícias confiáveis e ferramentas de verificação de fatos.
>     *   Promover a discussão sobre o papel da IA na sociedade e a importância da educação midiática.
> 
> 5.  **Hashtags:**
>     *   #IA #InteligenciaArtificial #Chatbots #Notícias #Informação #Jovens #GeraçãoZ #Tecnologia #FakeNews #PensamentoCrítico #ReutersInstitute #OpenAI #ChatGPT #Gemini
> 
> **Observações:**
> 
> *   Usar uma linguagem acessível e evitar jargões técnicos.
> *   Incluir imagens ou vídeos relevantes para tornar o post mais atraente.
> *   Monitorar os comentários e participar da discussão para promover o engajamento.
> *   Adaptar o post para diferentes plataformas de mídia social (Twitter, Instagram, Facebook, etc.).
> * O tema "Adoção de Chatbots de IA como Fonte de Informação, Principalmente entre Jovens" foi escolhido por apresentar maior relevância, novidade e impacto social em relação aos demais temas listados. Além de gerar discussões importantes sobre o futuro da informação e o papel da IA na sociedade.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Claro! Aqui está um rascunho de post para o Instagram da Alura, seguindo o seu plano:
> 
> ---
> 
> **[IMAGEM/CARROSSEL SUGERIDO: Poderia ser um carrossel. 1ª imagem: Título chamativo sobre um fundo moderno com ícones de IA e jovens. 2ª imagem: Gráfico simples mostrando a estatística dos 15%. 3ª imagem: Ícones representando vantagens (resumo, tradução, etc.). 4ª imagem: Ícone de alerta/lupa sobre um texto de IA. 5ª imagem: CTA com o logo da Alura.]**
> 
> ✨ **Jovens e IA: Uma Nova Era na Busca por Informação (Com um Alerta! ⚠️)** ✨
> 
> Onde você busca suas notícias hoje em dia? 🤔 Se você pensou "no Google", saiba que uma nova fonte está ganhando força, especialmente entre os mais jovens: a Inteligência Artificial! 🤖
> 
> Um estudo recente do Instituto Reuters revelou um dado impressionante: **15% dos jovens com menos de 25 anos já usam chatbots como o ChatGPT como sua principal fonte de informação.** Uma verdadeira revolução na forma como consumimos conteúdo!
> 
> **Por que essa galera está de olho na IA?** 🎯
> *   **Resumos na velocidade da luz:** Artigos longos viram pílulas de conhecimento.
> *   **Traduções instantâneas:** O mundo sem barreiras linguísticas.
> *   **Recomendações sob medida:** Conteúdo que realmente te interessa.
> *   **Respostas ágeis:** Dúvidas sobre atualidades? A IA responde!
> Uma mão na roda para estudantes e quem vive na correria, né? 🚀
> 
> **MAS, atenção! Nem tudo são flores...** 🥀
> Apesar da popularidade, a mesma pesquisa mostra que a **desconfiança ainda é grande**. Muitos usuários, incluindo os jovens, se preocupam com a confiabilidade das informações geradas por IA e ainda preferem notícias com supervisão humana.
> E o alerta se estende ao uso da IA generativa para disseminar informações, às vezes por figuras políticas, levantando debates sobre transparência.
> 
> Por isso, o **pensamento crítico** e a **checagem de fatos** são MAIS IMPORTANTES DO QUE NUNCA ao usar IA para se informar. Sempre questione e busque outras fontes! 🧐
> 
> Na Alura, acreditamos no poder da tecnologia para transformar, mas também na importância de usá-la com consciência e responsabilidade.
> 
> 🗣️ **E você, já usa chatbots para se manter informado(a)? Confia 100% nas respostas da IA? Conta pra gente nos comentários!** 👇
> 
> #InteligenciaArtificial #Chatbots #Jovens #Tecnologia
> 
> ---
> 
> **Observações sobre o rascunho:**
> *   **Tom:** Engajador, informativo e com um toque de alerta, adequado ao público da Alura.
> *   **Linguagem:** Simples e direta, com uso de emojis para aumentar o apelo visual e a informalidade controlada.
> *   **Estrutura:** Segue o plano fornecido, desde o título até a chamada para ação.
> *   **Conteúdo:** Cobre os pontos relevantes (crescimento, preferências, desconfiança, impacto político e o estudo).
> *   **CTA:** Incentiva a interação e sutilmente posiciona a Alura como uma fonte de conhecimento sobre o tema.
> *   **Hashtags:** Selecionei 4 relevantes e populares.
> 
> Espero que goste! 😊


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
